In [1]:
import sqlite3
from urllib.parse import urlparse, urlsplit
from hashlib import sha256 as hash


In [2]:
sqlite_file = 'C:/Users/zong/Documents/PowerShell/url_kb.sqlite3'

In [3]:
batch_size = 10000
with sqlite3.connect(sqlite_file) as conn:
    cur = conn.cursor()
    cur.execute("SELECT * FROM raw_url;")
    while True:
        all = cur.fetchmany(batch_size)
        if len(all) > 0:
            print(len(all))
        else:
            break
    cur.close()

10000
10000
10000
10000
10000
10000
10000
10000
10000


6017


In [4]:
with sqlite3.connect(sqlite_file) as conn:
    cur = conn.cursor()
    cur.execute("SELECT * FROM raw_url;")
    all = cur.fetchmany(10)
    cur.close()


In [5]:
t = all[0]
target_url = t[2]
target_url

'https://www.venea.net/'

In [6]:
urlparse(target_url)

ParseResult(scheme='https', netloc='www.venea.net', path='/', params='', query='', fragment='')

In [7]:
def create_table_parsed_urls():
    sql = """
    CREATE TABLE IF NOT EXISTS parsed_urls (
        hash text       NOT NULL PRIMARY KEY,
        scheme text     NOT NULL,
        netloc text     NOT NULL,
        path text       NOT NULL,
        params text     NOT NULL,
        query text      NOT NULL,
        fragment text   NOT NULL,
        count integer   
    	
    );
    """

    with sqlite3.connect(sqlite_file) as conn:
        try:
            cur = conn.cursor()
            cur.execute(sql)
            cur.close()
        except Error as e:
            print(e)

def insert_parsed_urls(sql_params):
    sql = ''' INSERT INTO parsed_urls(hash, scheme, netloc, path, params, query, fragment, count) VALUES (?,?,?,?,?,?,?,?)
    ON CONFLICT (hash) DO
    UPDATE 
    SET count = count + 1
    WHERE hash=?
    ;'''
    with sqlite3.connect(sqlite_file) as conn:
        try:
            cur = conn.cursor()
            cur.execute(sql, sql_params)
            cur.close()
            #conn.commit()
        except Exception as e:
            print(e)

def get_page(page_number):
    batch_size = 10000
    offset = (page_number - 1) * batch_size
    with sqlite3.connect(sqlite_file) as conn:
        cur = conn.cursor()
        cur.execute("SELECT * FROM raw_url LIMIT ? OFFSET ?;", (batch_size, offset))
        all = cur.fetchall()
        cur.close()
        return all

In [8]:
parse_result = urlparse(target_url)
print(parse_result)
print(parse_result.scheme)


h = hash(target_url.encode('UTF8')).hexdigest()
q = (h, parse_result.scheme, parse_result.netloc, parse_result.path, parse_result.params, parse_result.query, parse_result.fragment, 1, h)
q
insert_parsed_urls(q)


ParseResult(scheme='https', netloc='www.venea.net', path='/', params='', query='', fragment='')
https


In [19]:
records = get_page(3)
record_count = len(records)
if record_count > 0:
    for r in records:
        target_url = r[2]
        parse_result = urlparse(target_url)
        h = hash(target_url.encode('UTF8')).hexdigest()
        q = (h, parse_result.scheme, parse_result.netloc, parse_result.path, parse_result.params, parse_result.query, parse_result.fragment, 1, h)
        insert_parsed_urls(q)
else:
    print("do nothing")